In [ ]:
from espressomd import System, interactions, electrostatics
from espressomd.io.writer import vtf

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt


Theory: this notebook uses p3m, but the focus is on ion condensation rather than electrostatics methods. For the summer school, this will be the first tutorial using electrostatics so we might have to reuse some of the theory from 02-charged_system

In [ ]:

# Rod length
rod_length = 25.

# line charge density of the rod 
line_charge_dens =  2

# rod radius
rod_radius = 1.0

# Bjerrum length
bjerrum_length = 1.0

# ion diameter
ion_diameter = 1.0

# valency of the counterions
ion_valency = 1

# number of beads of the rod
N_rod_beads = 50


# accuracy of p3m algorithm 
p3m_accuracy =  1e-3;

In [ ]:
# particle types 
rod_type = 1
ion_type = 2

# total charge of the rod
total_rod_charge = line_charge_dens*rod_length

# charge per rod bead
rod_charge_per_bead = -total_rod_charge/N_rod_beads

# number of counterions
N_ions = int(total_rod_charge/ion_valency)

#TODO assert neutrality


system = System(box_l=3*[rod_length])

system.time_step = 0.01
system.cell_system.skin = 0.4

system.thermostat.set_langevin(kT=1.0, gamma=0.5, seed=3)



In [ ]:
wca_epsilon = 1.0

#ion-ion interaction
system.non_bonded_inter[ion_type,ion_type].wca.set_params(
      epsilon=wca_epsilon, sigma=ion_diameter)

# ion-rod interaction
system.non_bonded_inter[ion_type,rod_type].wca.set_params(
      epsilon=wca_epsilon, sigma=ion_diameter/2. + rod_radius)

In [ ]:
for idx in range(N_rod_beads):
    system.part.add(pos=[rod_length/2.,rod_length/2.,idx/N_rod_beads], type=rod_type, q=rod_charge_per_bead, fix=3*[True])
#TODO: comment on fix

for _ in range(N_ions):
    system.part.add(pos=np.random.random(3) * system.box_l, type=ion_type, q=ion_valency)


In [ ]:
force_cap=1
system.force_cap = force_cap

for i in range(4000):
    system.integrator.run(200)
    dist=system.analysis.min_dist(p1=[ion_type],p2=[ion_type, rod_type])
    print( f"Warm up step {i}: min_dist={dist} force_cap={force_cap}")
    force_cap = force_cap + 1.
    system.force_cap = force_cap
    
    if (dist >= ion_diameter):
        break

# Remove capping of LJ-interactions.
system.force_cap = 0


In [ ]:
p3m = electrostatics.P3M(prefactor=bjerrum_length, accuracy=p3m_accuracy)
system.actors.add(p3m)




In [ ]:
def integrate_calc_observables(system, N_frames, steps_per_frame, ion_type):
    energies = []
    radial_distances = []
    
    ions = system.part.select(type=ion_type)
    system_center = np.array(system.box_l)/2.
    
    for _ in range(N_frames):
        # run run the simulation for a few steps
        system.integrator.run(steps_per_frame)

        energies.append(system.analysis.energy()['total'])
        
        for ion in ions:
            radial_distances.append(np.linalg.norm(ion.pos_folded[0:2]-system_center[0:2]))
    
    return energies, radial_distances




In [ ]:
# run and look at energies to see how long it takes until equilibration and what the fluctuation timescale is

# MD frames to go
N_frames = 100

# number of timesteps per frame
steps_per_frame = 100

energies, distances = integrate_calc_observables(system,N_frames,steps_per_frame, ion_type)


In [ ]:
#save the positions to visualize
vsf_filename = f"ion_condensation.vsf"
vcf_filename = f"ion_condensation.vcf"
with open(vsf_filename, 'w') as vsf_file:
    vtf.writevsf(system, vsf_file, types='all')
with open(vcf_filename, 'w') as vcf_file:    
    vtf.writevcf(system, vcf_file, types='all')

In [ ]:
fig1 = plt.figure()
plt.plot(energies)
plt.xlabel('time frames')
plt.ylabel('system total energy')

In [ ]:
# cumulative histogram of the distances to the rod
# compare to PB?
# compare different charges on the rod?

log_bins=np.logspace(np.log10(0.8*rod_radius), np.log10(rod_length),num=50)
hist, bins = np.histogram(distances, bins=log_bins)
cum_hist = np.cumsum(hist)
cum_hist =cum_hist/cum_hist[-1]

fig2 = plt.figure()
plt.plot(log_bins[1:],cum_hist)
ax = plt.gca()
ax.set_xscale('log')
plt.xlabel('r')
plt.ylabel('P(r)')